# Test the ENC-DEC model

In [1]:
import numpy as np
import dataloader
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import models
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import os
import time
from torch_geometric.nn import GATv2Conv, global_mean_pool
# reload library
import importlib
import cv2
#import utils as ut
import pandas as pd
import DataDLC
from torch_geometric.data import Data, DataLoader
import tqdm
import time
import augmentation
import sklearn


# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import pickle as pkl


In [2]:
importlib.reload(dataloader)
importlib.reload(models)
importlib.reload(DataDLC)

<module 'DataDLC' from 'c:\\Users\\Usuario\\Documents\\Documents\\MVA\\Stage\\DLCProject\\Code\\GitHubRep\\Behavioral_Tagging_of_Mice_in_multiple_Mice_dataset_using_Deep_Learning\\src\\DataDLC.py'>

### Load data

---

**This is only to create an eassy graph to test the build_graph_2 function**



In [3]:
if False:
    data_dlc.coords = data_dlc.coords.iloc[range(3), :]
    data_dlc.n_frames = 3
    data_dlc.save(r'C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTestFormat\output1.h5')


## Create dataset

**Obs:** This is just a test to see if the model is working.

In [ ]:
# print numpy version
print(np.__version__)

# print pytorch version
print(torch.__version__)



In [ ]:
importlib.reload(dataloader)
importlib.reload(DataDLC)

In [40]:

dataloader.reload_module()

In [41]:
# deactivate warnings
if False:
    import warnings
    warnings.filterwarnings("ignore")


    data_loader = dataloader.DLCDataLoader(r'C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTrainTest', batch_size=1, num_workers=0, device='cpu', window_size=3, stride = 1, build_graph=True)
    #data_loader = dataloader.DLCDataLoader(r'C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTestDataset', batch_size=1, num_workers=0, device='cpu', window_size=3, stride = 1, build_graph=True, behaviour='contacts generaux (R + V) active')
    

---

## Load Dataset

*Graphs were already created and saved in the data folder*

In [ ]:
del dataset

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load the data
dataset = torch.load(r'c:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\datasets\new_entire_dataset.pkl', map_location=device)


In [ ]:
print('The dataset has %d samples' % len(dataset))

In [7]:
# Select the behaviour to classify (Grooming in this case)
indx_behaviour1 = 0
indx_behaviour2 = 9


#### Split Train-Test

**Shuffle**

In [8]:
# Suffle the dataset
np.random.seed(0)
np.random.shuffle(dataset)


In [9]:
# Train / Test split
train_size = int(0.8 * len(dataset))

train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:]

In [ ]:
print(len(train_dataset))
print(len(test_dataset))

In [ ]:
dataset[0].behaviour_names[indx_behaviour1]

In [ ]:
# Class balance in the training set
behaviour = [sample.behaviour[indx_behaviour1] for sample in train_dataset]
behaviour = np.array(behaviour)
print('The class balance in the training set is:')
print(np.unique(behaviour, return_counts=True))

plt.hist(behaviour, bins=2)
plt.title('Class balance in the training set')
plt.show()

#### Augmentation

In [ ]:
print('Merging the behaviours')
augmentation.merge_symetric_behaviours_version2(indx_behaviour1, indx_behaviour2, train_dataset)
print('Generating rotation augmentation')
# Rotate the dataset
augmentation.rotate_samples(train_dataset, indx_behaviour1)
#print('Downsampling the inactive behaviours')
#train_dataset = augmentation.downsample_majority_class(train_dataset, indx_behaviour1)
 

In [ ]:
                                                
for i in range(len(train_dataset)):
    train_dataset[i].behaviour = train_dataset[i].behaviour[indx_behaviour1]
for i in range(len(test_dataset)):
    test_dataset[i].behaviour = test_dataset[i].behaviour[indx_behaviour1]
print('Done selecting the behaviour')


In [ ]:
print('The train dataset has %d samples' % len(train_dataset))
print('The test dataset has %d samples' % len(test_dataset))


In [ ]:
# Class balance in the training set
behaviour = [sample.behaviour for sample in train_dataset]
behaviour = np.array(behaviour)
print('The class balance in the training set is:')
print(np.unique(behaviour, return_counts=True))

plt.hist(behaviour, bins=2)
plt.title('Class balance in the training set')
plt.show()


In [15]:
# Compute weights for the loss function


weights = sklearn.utils.class_weight.compute_class_weight('balanced', classes = np.unique(behaviour), y = behaviour)

In [ ]:
print('The weights for the loss function are:')
print(weights)

In [ ]:
batch_size = 32

# Create the dataloaders for train, validation and test
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


---

### Model definition

In [ ]:
importlib.reload(models)

In [22]:
# Define the model
graphencoder = models.GATEncoder(nout = 64, nhid=32, attention_heads = 2, n_in = 4, n_layers=4, dropout=0.2)
class_head = models.ClassificationHead(n_latent=64, nhid = 32, nout = 2)


In [ ]:
model = models.GraphClassifier(graphencoder, class_head, readout = 'mean')

model.to(device)

print('The model has %d trainable parameters' % sum(p.numel() for p in model.parameters() if p.requires_grad))


---

In [24]:
def mean_pooling_per_graph(embbed, batch, frame_mask):
    ''' Mean pooling of the embeddings per graph, only the central frame '''
    out = []
    for i in range(batch.max()+1):
        out.append(embbed[batch==i][frame_mask[batch==i] == frame_mask[batch==i].median()].mean(dim=0))
    return torch.stack(out)



In [25]:
batch_0 = next(iter(train_loader))

In [ ]:
batch_0

In [28]:
embed = graphencoder(batch_0.x, batch_0.edge_index)

In [ ]:
embed.shape

In [30]:
lat = mean_pooling_per_graph(embed, batch_0.batch, batch_0.frame_mask)

In [ ]:
lat.shape

In [ ]:
out = class_head(lat)

In [ ]:
out.shape

----

### Loop

In [33]:
# Directory to save the model checkpoints
checkpoint_dir = r'./deletefolder/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

def save_checkpoint(model, optimizer, epoch, loss, path):
    # Save the model, optimizer state, epoch, and loss
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, path)
    print(f"Checkpoint saved at {path}")

def load_checkpoint(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint loaded from {path}, at epoch {epoch}")
    return epoch

In [34]:
# Import f1_score and matthews_corrcoef from sklearn
from sklearn.metrics import f1_score, matthews_corrcoef

In [35]:
num_epochs = 200
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
metric_0 = f1_score
metric_1 = matthews_corrcoef

if True:
    # Training loop
    actual_epoch = 0
    #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    
    

if False:
    # Load the model from a checkpoint
    checkpoint_path = r'/gpfs/users/alvarezj/workdir/Project/Data/Checkpoints/GAT_Sniffin_Anal/checkpoint_epoch_170.pth'
    actual_epoch = load_checkpoint(model, optimizer, checkpoint_path)


In [36]:
criterion = nn.CrossEntropyLoss(weight=torch.tensor(weights, dtype=torch.float32).to(device))
writer = SummaryWriter(log_dir='./deletefolder/runs/GAT_Following_w11_new_encoder_2')  # TensorBoard writer


In [ ]:
start_time = time.time()  # Time the training
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    correct_class_0 = 0
    correct_class_1 = 0
    total_class_0 = 0
    total_class_1 = 0
    total = 0
    i = 0

    for data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(data)
        labels = data.behaviour

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predicted = outputs.argmax(dim=1)
        correct_class_0 += (predicted[labels == 0] == labels[labels == 0]).sum().item()
        correct_class_1 += (predicted[labels == 1] == labels[labels == 1]).sum().item()
        total_class_0 += (labels == 0).sum().item()
        total_class_1 += (labels == 1).sum().item()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Log training loss and accuracy at each step
        if i % 100 == 0:  # Log every 100 iterations, adjust as needed
            writer.add_scalar('Loss/Train', loss.item(), (actual_epoch + epoch) * len(train_loader) + i)
            writer.add_scalar('Accuracy/Train', correct / total, (actual_epoch + epoch) * len(train_loader) + i)
            # Metrics 
            writer.add_scalar('Metrics/Train/F1', metric_0(labels, predicted), (actual_epoch + epoch) * len(train_loader) + i)
            writer.add_scalar('Metrics/Train/Matthews', metric_1(labels, predicted), (actual_epoch + epoch) * len(train_loader) + i)
        i += 1

    train_accuracy = correct / total
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + actual_epoch + 1}, Training Loss: {avg_train_loss}, Training Accuracy: {train_accuracy}")

    # Validation phase
    model.eval()
    val_loss = 0
    correct = 0
    correct_class_0 = 0
    correct_class_1 = 0
    total_class_0 = 0
    total_class_1 = 0
    total = 0
    with torch.no_grad():
        for val_data in tqdm.tqdm(test_loader):
            val_outputs = model(val_data)
            val_labels = val_data.behaviour
            val_loss += criterion(val_outputs, val_labels).item()
            val_predicted = val_outputs.argmax(dim=1)
            correct_class_0 += (val_predicted[val_labels == 0] == val_labels[val_labels == 0]).sum().item()
            correct_class_1 += (val_predicted[val_labels == 1] == val_labels[val_labels == 1]).sum().item()
            total_class_0 += (val_labels == 0).sum().item()
            total_class_1 += (val_labels == 1).sum().item()
            correct += (val_predicted == val_labels).sum().item()
            total += val_labels.size(0)

    val_accuracy = correct / total
    avg_val_loss = val_loss / len(test_loader)

    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy}")

    # Log validation metrics
    writer.add_scalar('Loss/Validation', avg_val_loss, actual_epoch + epoch)
    writer.add_scalar('Accuracy/Validation', val_accuracy, actual_epoch + epoch)
    writer.add_scalar('Accuracy/Avarage_inactive_class_Validation', correct_class_0 / total_class_0, actual_epoch + epoch)
    writer.add_scalar('Accuracy/Avarage_active_class_Validation', correct_class_1 / total_class_1, actual_epoch + epoch)
    writer.add_scalar('Accuracy/Average_per_class_Validation', ((correct_class_0 / total_class_0) + (correct_class_1 / total_class_1)) / 2, actual_epoch + epoch)
    # Metrics
    writer.add_scalar('Metrics/Validation/F1', metric_0(val_labels, val_predicted), actual_epoch + epoch)
    writer.add_scalar('Metrics/Validation/Matthews', metric_1(val_labels, val_predicted), actual_epoch + epoch)

    # Step the scheduler
    #scheduler.step()

    # Log learning rate
    current_lr = optimizer.param_groups[0]['lr']
    writer.add_scalar('Learning Rate', current_lr, actual_epoch + epoch)
    print(f"Learning Rate after epoch {epoch + 1}: {current_lr}")

    # Save checkpoint after each 5 epochs
    if (epoch + 1) % 10 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch + actual_epoch + 1}.pth')
        save_checkpoint(model, optimizer, epoch + 1, avg_train_loss, checkpoint_path)

# Save the final model
if num_epochs % 5 != 0:
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch + actual_epoch + 1}.pth')
    save_checkpoint(model, optimizer, epoch + 1, avg_train_loss, checkpoint_path)

# Time the training
end_time = time.time()
print(f"Training took {end_time - start_time} seconds, for {num_epochs} epochs")
# Close the TensorBoard writer
writer.close()

print("Training finished")

## Benchmark model 

In [116]:
batch = next(iter(train_loader))

x, edge_index, batch, frame_mask = batch.x, batch.edge_index, batch.batch, batch.frame_mask

In [ ]:
frame_mask

In [145]:

model = models.SimpleMLPforGraph(n_in=144, n_hid=128, n_out=2)

# Trainning

In [146]:
# Trainning loop
num_epochs = 10
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter(log_dir='runs/gcn_action_detection')  # TensorBoard writer


In [ ]:

# Training Loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for i, data in tqdm.tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        outputs = model(data).unsqueeze(0)
        labels = data.y
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        train_loss += loss.item()
        predicted = outputs.argmax(dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        # Log training loss and accuracy at each step
        if i % 10 == 0:  # Log every 10 iterations, adjust as needed
            writer.add_scalar('Loss/Train', loss.item(), epoch * len(train_loader) + i)
            writer.add_scalar('Accuracy/Train', correct / total, epoch * len(train_loader) + i)
            #print(f"Epoch {epoch+1}, Step {i}, Loss: {loss.item()}, Accuracy: {correct / total}")

    train_accuracy = correct / total
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss}, Training Accuracy: {train_accuracy}")

    # Validation phase
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for val_data in tqdm.tqdm(test_loader):
            val_outputs = model(val_data).unsqueeze(0)
            val_labels = val_data.y
            val_loss += criterion(val_outputs, val_labels).item()
            val_predicted = val_outputs.argmax(dim=1)
            correct += (val_predicted == val_labels).sum().item()
            total += val_labels.size(0)
    
    val_accuracy = correct / total
    avg_val_loss = val_loss / len(test_loader)
    
    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy}")

    # Log validation metrics
    writer.add_scalar('Loss/Validation', avg_val_loss, epoch)
    writer.add_scalar('Accuracy/Validation', val_accuracy, epoch)

# Close the TensorBoard writer
writer.close()


In [ ]:
    for i, data in tqdm.tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        outputs = model(data)
        labels = data.y
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        train_loss += loss.item()
        predicted = outputs.argmax(dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
        # Log training loss and accuracy at each step
        if i % 10 == 0:  # Log every 10 iterations, adjust as needed
            writer.add_scalar('Loss/Train', loss.item(), epoch * len(train_loader) + i)
            writer.add_scalar('Accuracy/Train', correct / total, epoch * len(train_loader) + i)
            #print(f"Epoch {epoch+1}, Step {i}, Loss: {loss.item()}, Accuracy: {correct / total}")

    train_accuracy = correct / total
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss}, Training Accuracy: {train_accuracy}")

    # Validation phase
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for val_data in test_loader:
            val_outputs = model(val_data)
            val_labels = val_data.y
            val_loss += criterion(val_outputs, val_labels).item()
            val_predicted = val_outputs.argmax(dim=1)
            correct += (val_predicted == val_labels).sum().item()
            total += val_labels.size(0)
    
    val_accuracy = correct / total
    avg_val_loss = val_loss / len(test_loader)
    
    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy}")

    # Log validation metrics
    writer.add_scalar('Loss/Validation', avg_val_loss, epoch)
    writer.add_scalar('Accuracy/Validation', val_accuracy, epoch)

# Close the TensorBoard writer
writer.close()


# Trainning loop
epochs = 50
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    running_loss = 0.0
    las
    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        out = model.forward(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100
        

In [ ]:
out = GraphAE.forward(data_loader.dataset[0].x, data_loader.dataset[0].edge_index, data_loader.dataset[0].frame_mask)

In [ ]:
# Loss and optimizer
loss = GraphAE.loss(data_loader.dataset[0].x, out)

In [ ]:
print(loss)

# Trainning loop

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
encoder = models.GATEncoder(nout = 64, nhid=16, attention_hidden=2, n_in=3, dropout=0.5).to(device)
print(encoder)
decoder = models.GATDecoder(n_latent=64, n_hidden=16, n_out=3).to(device)
print(decoder)
model = models.GraphAE(encoder, decoder).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
data = data_loader.dataset

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    for i in range(len(data)):

        out = model(data[i].x.to(device), data[i].edge_index.to(device), data[i].frame_mask.to(device))
        loss = model.loss(data[i].x, out)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss {loss.item()}')

model.eval()
out = model(data[0].x, data[0].edge_index, data[0].frame_mask)

In [ ]:
print(out[0][0].shape)

In [ ]:
numpyout = out[0][0].detach().numpy()

In [ ]:
# Plot the output of the model, the first dimension are the points, and the second one is the x and y coordinates
plt.scatter(numpyout[:,0], numpyout[:,1])


In [ ]:
print(data_loader.dataset[0].x.shape)
print(x[0].shape)
print(x[1].shape)

In [ ]:
print(data_loader.dataset[0].x.shape)
print(x[0].shape)
print(x[1].shape)

In [ ]:
contacts = behaviour.iloc[:,1]

---

## Test the new model

In [25]:
import numpy as np
import dataloader
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import models
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import os
import time
from torch_geometric.nn import GATv2Conv, global_mean_pool
# reload library
import importlib
import cv2
#import utils as ut
import pandas as pd
import DataDLC
from torch_geometric.data import Data, DataLoader, DataListLoader
import tqdm
import time
import augmentation
import sklearn
# Import f1_score and matthews_corrcoef from sklearn
from sklearn.metrics import f1_score, matthews_corrcoef



# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import pickle as pkl



In [120]:
importlib.reload(augmentation)

<module 'augmentation' from 'c:\\Users\\Usuario\\Documents\\Documents\\MVA\\Stage\\DLCProject\\Code\\GitHubRep\\Behavioral_Tagging_of_Mice_in_multiple_Mice_dataset_using_Deep_Learning\\src\\augmentation.py'>

In [84]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

torch.cuda.empty_cache()

# Load the data
dataset = torch.load(r'c:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\datasets\dataset_w1.pkl', map_location=device)


cpu


C:\Users\Usuario\AppData\Local\Temp\ipykernel_15436\1586004789.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(r'c:\Users\Usuario\Documents\Document

In [121]:

#Index(['General_Contacts', 'Sniffing_R', 'Sniffing_head_R', 'Sniffing_other_R',
    #    'Sniffing_anal_R', 'Poursuit_R', 'Dominance_R', 'Rearing_R',
    #    'Grooming_R', 'Sniffing_V', 'Sniffing_head_V', 'Sniffing_other_V',
    #    'Sniffing_anal_V', 'Poursuit_V', 'Dominance_V', 'Rearing_V',
    #    'Grooming_V'],
    #   dtype='object')
# Select the behaviour to classify (Grooming in this case)
indx_behaviour1 = 1
indx_behaviour2 = 6

# Select the behaviour to classify
name_behaviour = dataset[0].behaviour_names[indx_behaviour1]


In [122]:


#print('Selecting the behaviour')
#for i in range(len(dataset)):
#    dataset[i].behaviour = dataset[i].behaviour[indx_behaviour1]

window_size = 5
seq_dataset = dataloader.SequenceDataset(dataset, sequence_length=window_size)

def split_dataset(dataset, split_ratio=0.8):
    dataset.shuffle()  # Shuffle the sequences
    split_idx = int(len(dataset) * split_ratio)
    train_data = dataset.sequences[:split_idx]
    test_data = dataset.sequences[split_idx:]
    return train_data, test_data

# Split the dataset into train and test sets
train_data, test_data = split_dataset(seq_dataset, split_ratio=0.8)



In [123]:


print('The train dataset has %d samples' % len(train_data))
print('The test dataset has %d samples' % len(test_data))

The train dataset has 124219 samples
The test dataset has 31055 samples


In [124]:
seq = train_data[0]

In [125]:
seq[1][indx_behaviour1]

tensor(1)

In [126]:
augmentation.merge_symetric_behaviours_sequences(indx_behaviour1, indx_behaviour2, train_data)

In [ ]:
indices = []
for i in range(len(dataset)):
    if dataset[i].behaviour[behaviour] == torch.tensor(1):
        indices.append(i)
# Symetric wrt y axis
for indx in indices:
    new_sample = dataset[indx].clone()
    new_sample.x[:,0] = torch.tensor(1) - new_sample.x[:,0]
    dataset.append(new_sample)
# Symetric wrt x axis
for indx in indices:
    new_sample = dataset[indx].clone()
    new_sample.x[:,1] = torch.tensor(1) - new_sample.x[:,1]
    dataset.append(new_sample)
# Transpose
for indx in indices:
    new_sample = dataset[indx].clone()
    new_sample.x[:,0], new_sample.x[:,1] = new_sample.x[:,1], new_sample.x[:,0]
    dataset.append(new_sample)
# Rotate 180 degrees
for indx in indices:
    new_sample = dataset[indx].clone()
    new_sample.x[:,0] = torch.tensor(1) - new_sample.x[:,0]
    new_sample.x[:,1] = torch.tensor(1) - new_sample.x[:,1]
    dataset.append(new_sample)
return


In [96]:
#print('Merging the behaviours')
#augmentation.merge_symetric_behaviours_version2(indx_behaviour1, indx_behaviour2, train_dataset)
#print('Generating rotation augmentation')
# Rotate the dataset
#augmentation.rotate_samples(train_dataset, indx_behaviour1)
#print('Downsampling the inactive behaviours')
#train_dataset = augmentation.downsample_majority_class(train_dataset, indx_behaviour1)
                                                 
# #for i in range(len(train_dataset)):
#     train_dataset[i].behaviour = train_dataset[i].behaviour[indx_behaviour1]
# for i in range(len(test_dataset)):
#     test_dataset[i].behaviour = test_dataset[i].behaviour[indx_behaviour1]
# print('Done selecting the behaviour')


In [127]:


print('The train dataset has %d samples' % len(train_data))
print('The test dataset has %d samples' % len(test_data))




The train dataset has 132973 samples
The test dataset has 31055 samples


In [ ]:

# behaviour = [sample.behaviour for sample in train_dataset]
# behaviour = np.array(behaviour)

# weights_ = sklearn.utils.class_weight.compute_class_weight('balanced', classes = np.unique(behaviour), y = behaviour)
# del behaviour

batch_size = 10

# Create the dataloaders for train, validation and test
train_loader = DataListLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataListLoader(test_data, batch_size=batch_size, shuffle=True)

# Define the model
model = models.GAT_LSTM(input_dim = 4, hidden_dim=64, lstm_hidden_dim=128, num_classes=1, num_nodes=36, heads=4)

model.to(device)

print('The model has %d trainable parameters' % sum(p.numel() for p in model.parameters() if p.requires_grad))


# Directory to save the model checkpoints
checkpoint_dir = r'/gpfs/users/alvarezj/workdir/Project/Data/Checkpoints/GAT_LSTM/%s' % name_behaviour
os.makedirs(checkpoint_dir, exist_ok=True)

def save_checkpoint(model, optimizer, epoch, loss, path):
    # Save the model, optimizer state, epoch, and loss
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, path)
    print(f"Checkpoint saved at {path}")

def load_checkpoint(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint loaded from {path}, at epoch {epoch}")
    return epoch

num_epochs = 200
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)



if True:
    # Training loop
    actual_epoch = 0
    #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    
    

if False:
    # Load the model from a checkpoint
    checkpoint_path = r'/gpfs/users/alvarezj/workdir/Project/Data/Checkpoints/GAT_Sniffin_Anal/checkpoint_epoch_170.pth'
    actual_epoch = load_checkpoint(model, optimizer, checkpoint_path)

#y = [d.behaviour.cpu().detach().numpy() for d in dataset]
#y = np.array(y)
#weight_ = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y), y)
#weight_ = torch.tensor([0.55852461, 4.7717076]).to(device) # Grooming (before adding last analysis)
#weight_ = torch.tensor([0.76221058, 1.45343216]).to(device) # Dominance (before adding last analysis)
#weight_ = torch.tensor([0.58626763, 3.39795837]).to(device) # Rearing (before adding last analysis)
#weight_ = torch.tensor([0.60796284, 2.81561145]).to(device) # Sniffing anal
#weight_ = torch.tensor([0.66321035, 2.03176556]).to(device) # Following with only rotation augmentation
#weight_ = torch.tensor([0.5331574 , 8.03979407]).to(device) # Following with merged behaviours only
#print(' The class weights are: ', weight_)
# Destroy y
#criterion = nn.CrossEntropyLoss(weight=torch.tensor(weights_, dtype=torch.float32).to(device))
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy with logits
#criterion = nn.CrossEntropyLoss()
name_file = datetime.now().strftime('%b%d_%H-%M-%S')
writer = SummaryWriter(log_dir=os.path.join('runs/GAT_LSTM/%s/' % name_behaviour, name_file))

f = open(os.path.join('runs/GAT_LSTM/%s/' % name_behaviour, name_file, 'info.txt'), 'w')
f.write('Dataset used: new_dataset_large.pkl\n')
f.write('Behaviour selected: %s\n' % name_behaviour)
f.write('Number of epochs: %d\n' % num_epochs)
f.write('Batch size: %d\n' % batch_size)
f.write('Learning rate: %f\n' % lr)
f.write('Window size: %d\n' % window_size)
f.write('Model used:%s\n' % str(model))
f.write('Number of trainable parameters: %d\n' % sum(p.numel() for p in model.parameters() if p.requires_grad))
f.write('Criterion: BCEWithLogitsLoss\n')
f.write('Optimizer: Adam\n')
f.write('Scheduler: None\n')
f.write('Data augmentation: None\n')
#f.write('Class weights: %s\n' % str(weights_))
f.close()


start_time = time.time()  # Time the training
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    batch_idx = 0
    running_loss = 0.0

    # Loop over each batch from the DataListLoader
    for batch in tqdm.tqdm(train_loader):
        optimizer.zero_grad()  # Zero the gradients

        # Unpack batch (sequence and label)
        sequences, labels = zip(*batch)  # batch is a list of (sequence, label) tuples
        
        # Forward pass through the model
        output = model(sequences)

        # Assuming labels are already tensors, no need to convert them again
        labels = torch.tensor(labels, dtype=torch.float32).to(device).unsqueeze(1)  # Add singleton dimension for BCELoss

        # Compute the loss
        loss = criterion(output, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track the running loss
        running_loss += loss.item()

        # Predictions and accuracy computation
        predictions = torch.sigmoid(output).round()  # Convert logits to probabilities and round for binary
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

        if batch_idx % 100 == 0:  # Log every 100 iterations, adjust as needed
            # Log metrics to TensorBoard after each batch
            writer.add_scalar('Loss/train', loss.item(), epoch * len(train_loader) + batch_idx)
            writer.add_scalar('Accuracy/train', correct / total * 100, epoch * len(train_loader) + batch_idx)

        batch_idx += 1

    # Print epoch loss and training accuracy
    epoch_loss = running_loss / len(train_loader)
    accuracy_train = correct / total * 100
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {accuracy_train:.2f}%')

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    val_correct_class_0 = 0
    val_correct_class_1 = 0
    val_total_class_0 = 0
    val_total_class_1 = 0
    val_all_labels = []
    val_all_predictions = []

    with torch.no_grad():  # Disable gradient calculation for validation
        for val_batch in tqdm.tqdm(test_loader):
            # Unpack validation batch
            val_sequences, val_labels = zip(*val_batch)  # batch is a list of (sequence, label) tuples
            
            # Forward pass through the model
            val_output = model(val_sequences)

            # Convert labels to tensor
            val_labels = torch.tensor(val_labels, dtype=torch.float32).to(device).unsqueeze(1)  # Add singleton dimension for BCELoss

            # Compute the loss
            val_loss = criterion(val_output, val_labels)
            val_running_loss += val_loss.item()

            # Predictions and accuracy computation
            val_predictions = torch.sigmoid(val_output).round()  # Convert logits to probabilities and round for binary
            val_correct += (val_predictions == val_labels).sum().item()
            val_total += val_labels.size(0)
            val_correct_class_0 += (val_predictions[val_labels == 0] == val_labels[val_labels == 0]).sum().item()
            val_correct_class_1 += (val_predictions[val_labels == 1] == val_labels[val_labels == 1]).sum().item()
            val_total_class_0 += (val_labels == 0).sum().item()
            val_total_class_1 += (val_labels == 1).sum().item()

            # Store predictions and labels for metrics
            val_all_labels.extend(val_labels.cpu().numpy())
            val_all_predictions.extend(val_predictions.cpu().numpy())
        

    # Calculate validation metrics
    f1_val = f1_score(val_all_labels, val_all_predictions)
    mcc_val = matthews_corrcoef(val_all_labels, val_all_predictions)

    # Print validation metrics
    val_epoch_loss = val_running_loss / len(test_loader)
    accuracy_val = val_correct / val_total * 100
    print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {accuracy_val:.2f}%, F1 Score: {f1_val:.4f}, MCC: {mcc_val:.4f}')

    # Log validation metrics to TensorBoard
    writer.add_scalar('Loss/validation', val_epoch_loss, epoch)
    writer.add_scalar('Accuracy/validation', accuracy_val, epoch)
    writer.add_scalar('Accuracy/Avarage_inactive_class_validation', val_correct_class_0 / val_total_class_0, epoch)
    writer.add_scalar('Accuracy/Avarage_active_class_validation', val_correct_class_1 / val_total_class_1, epoch)
    writer.add_scalar('Accuracy/Average_per_class_validation', ((val_correct_class_0 / val_total_class_0) + (val_correct_class_1 / val_total_class_1)) / 2, epoch)
    writer.add_scalar('F1 Score/validation', f1_val, epoch)
    writer.add_scalar('MCC/validation', mcc_val, epoch)

    # Step the scheduler
    #scheduler.step()

    # # Log learning rate
    # current_lr = optimizer.param_groups[0]['lr']
    # writer.add_scalar('Learning Rate', current_lr, actual_epoch + epoch)
    # print(f"Learning Rate after epoch {epoch + 1}: {current_lr}")

    # Save checkpoint after each 5 epochs
    if (epoch + 1) % 10 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch + actual_epoch + 1}.pth')
        save_checkpoint(model, optimizer, epoch + 1, epoch_loss, checkpoint_path)

# Save the final model
if num_epochs % 10 != 0:
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch + actual_epoch + 1}.pth')
    save_checkpoint(model, optimizer, epoch + 1, epoch_loss, checkpoint_path)

# Time the training
end_time = time.time()
print(f"Training took {end_time - start_time} seconds, for {num_epochs} epochs")
# Close the TensorBoard writer
writer.close()

print("Training finished")